# III Neuroniniai tinklai (PyTorch)

In [ ]:
import time

import torch
import torch.nn.functional as F

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
DEVICE     = ('cuda:0' if torch.cuda.is_available() else 'cpu')
NUM_EPOCHS = 250

In [ ]:
data = pd.read_csv('xor.csv')
X    = data[['x1', 'x2']].values
y    = data['label'].values

In [ ]:
plt.figure(figsize=(14,8))
plt.scatter(X[y==0, 0], X[y==0, 1], marker='o', s=80, c='b')
plt.scatter(X[y==1, 0], X[y==1, 1], marker='o', s=80, c='g')
plt.tight_layout()

In [ ]:
class MLPLinear(torch.nn.Module):

    def __init__(self, num_features, num_hidden_1, num_classes):
        super(MLPLinear, self).__init__()
        
        self.num_classes = num_classes
        
        self.linear_1   = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_out = torch.nn.Linear(num_hidden_1, num_classes)
        
    def forward(self, x):
        
        out = self.linear_1(x)
        #out = F.relu(out)
        
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [ ]:
torch.manual_seed(RANDOM_SEED)

model     = MLPLinear(num_features=2, num_hidden_1=50, num_classes=2)
model     = model1.to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [ ]:
start_time     = time.time()
minibatch_cost = []

features = torch.tensor(X, dtype=torch.float).to(DEVICE)
targets  = torch.tensor(y, dtype=torch.long).to(DEVICE)

for epoch in range(NUM_EPOCHS):
    logits, probas = model(features)
    cost           = F.cross_entropy(logits, targets)
   
    optimizer.zero_grad()
    cost.backward()
    minibatch_cost.append(cost)
    optimizer.step()

    print (f'epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | cost: {cost:.4f}')
    
print(f'laikas:{((time.time() - start_time)/60)} min')

In [ ]:
def plot_decision_regions(X, y, classifier, resolution=0.02):

    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    tensor = torch.tensor(np.array([xx1.ravel(), xx2.ravel()]).T).float()
    logits, probas = classifier.forward(tensor)
    Z = np.argmax(probas.detach().numpy(), axis=1)

    Z = Z.reshape(xx1.shape)
    
    plt.figure(figsize=(14,8))
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, color=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)

In [ ]:
plot_decision_regions(features, targets, classifier=model1)
plt.tight_layout()